In [22]:
import BNSimpleReduction as BNred
import time
import os
import cana
import time
import networkx as nx
import matplotlib.pyplot as plt
from cana.datasets.bio import THALIANA

In [ ]:

startTime = time.time()
# BNred.main(Parameter_1, Parameter_2)
# Parameter_1: Boolean network file
# Parameter_2: Desired fixed point attractor (steady state) in the network
BNred.main("./networks/t_cell.txt", "01111001111001011101011")
endTime = time.time() - startTime
print(endTime)


In [24]:
def perturb_network_and_save(input_file, node_name, state='ON'):
    # perturbation 논리 생성
    if state.upper() == 'ON':
        new_logic = f"{node_name} = {node_name} or !{node_name}"
    elif state.upper() == 'OFF':
        new_logic = f"{node_name} = {node_name} & !{node_name}"
    else:
        raise ValueError("State must be 'ON' or 'OFF'")

    with open(input_file, 'r') as f:
        lines = f.readlines()

    updated_lines = []
    replaced = False

    for line in lines:
        if line.strip().startswith(f"{node_name} ="):
            updated_lines.append(new_logic + '\n')
            replaced = True
        else:
            updated_lines.append(line)

    if not replaced:
        raise ValueError(f"Node '{node_name}' not found in the network.")

    # 파일 이름 구성
    base_name = os.path.splitext(os.path.basename(input_file))[0]
    output_file = f"{base_name}_{node_name}_{state.upper()}.txt"

    with open(output_file, 'w') as f:
        f.writelines(updated_lines)

    print(f"📄 Perturbed network saved to: {output_file}")
    return output_file

In [25]:
perturb_network_and_save('networks/grieco.txt', 'JUN', state='OFF')

📄 Perturbed network saved to: grieco_JUN_OFF.txt


'grieco_JUN_OFF.txt'

In [15]:
def update_attractor_state_verbose(original_state, input_file, node_name, state='ON'):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # 노드 순서 추출
    node_order = [line.split('=')[0].strip() for line in lines]

    # 인덱스 확인
    if node_name not in node_order:
        raise ValueError(f"Node '{node_name}' not found in the network.")
    
    idx = node_order.index(node_name)

    # 상태값 설정
    state_value = '1' if state.upper() == 'ON' else '0'
    updated_state = list(original_state)

    if len(updated_state) != len(node_order):
        raise ValueError("Length mismatch between attractor string and number of nodes.")

    # 변경 적용
    old_value = updated_state[idx]
    updated_state[idx] = state_value

    # 출력
    print(f"{node_name}은(는) {idx}번째 노드이며, attractor 상태를 {old_value} → {state_value}로 변경했습니다.")

    return ''.join(updated_state)

In [17]:
original_state = "01101010100100001011111111001111100110110110101111110"
updated_state = update_attractor_state_verbose(original_state, 'networks/grieco.txt', 'JUN', state='OFF')
print(updated_state)

JUN은(는) 23번째 노드이며, attractor 상태를 1 → 0로 변경했습니다.
01101010100100001011111011001111100110110110101111110


In [ ]:
01101010100100001011111111001111100110110110101111110
01101010100100001011111011001111100110110110101111110

In [29]:
import os

def full_perturbation_pipeline(network_path, original_state, mutated_node, mutated_state):
    # 상태값 설정
    if mutated_state.upper() == 'ON':
        logic = f"{mutated_node} = {mutated_node} or !{mutated_node}"
        state_bit = '1'
    elif mutated_state.upper() == 'OFF':
        logic = f"{mutated_node} = {mutated_node} & !{mutated_node}"
        state_bit = '0'
    else:
        raise ValueError("mutated_state must be 'ON' or 'OFF'")

    # 원본 네트워크 불러오기
    with open(network_path, 'r') as f:
        lines = f.readlines()

    node_order = [line.split('=')[0].strip() for line in lines]
    if mutated_node not in node_order:
        raise ValueError(f"Node '{mutated_node}' not found in network.")
    idx = node_order.index(mutated_node)

    # attractor 문자열 업데이트
    updated_state = list(original_state)
    old_val = updated_state[idx]
    updated_state[idx] = state_bit
    updated_state_str = ''.join(updated_state)
    print(f"📌 {mutated_node}은(는) {idx}번째 노드이며, attractor 상태를 {old_val} → {state_bit}로 변경했습니다.")

    # perturbed 네트워크 파일명 만들기
    network_dir = os.path.dirname(network_path)
    base_name = os.path.splitext(os.path.basename(network_path))[0]
    perturbed_name = f"{base_name}_{mutated_node}_{mutated_state.upper()}.txt"
    perturbed_path = os.path.join(network_dir, perturbed_name)

    # perturbed 네트워크 저장
    updated_lines = []
    for line in lines:
        if line.strip().startswith(f"{mutated_node} ="):
            updated_lines.append(logic + '\n')
        else:
            updated_lines.append(line)

    with open(perturbed_path, 'w') as f:
        f.writelines(updated_lines)
    print(f"📄 Perturbed network saved to: {perturbed_path}")

    # BNred.main에 넘겨줄 경로 확인용 문자열 출력
    print(f"\n🔧 아래를 BNred.main()에 사용하세요:")
    print(f"BNred.main(\"{perturbed_path}\", \"{updated_state_str}\")")

    # 결과 폴더 경로
    parent_dir = os.path.dirname(network_dir)
    result_folder = os.path.join(parent_dir, 'results', f"{base_name}_{mutated_node}_{mutated_state.upper()}_{updated_state_str}")
    model_path = os.path.join(result_folder, "OnePlusProductNet.txt")

    # 결과 파일 읽기
    if not os.path.exists(model_path):
        print(f"❗ 아직 결과 파일이 없습니다: {model_path}")
        return None, updated_state_str, perturbed_path

    with open(model_path, 'r') as f:
        modeltext = f.read()

    print(f"📥 modeltext 로드 완료: {model_path}")
    return modeltext, updated_state_str, perturbed_path

In [30]:
network = "networks/grieco.txt"
attractor = "01101010100100001011111111001111100110110110101111110"
node = "JUN"
state = "OFF"

In [31]:

modeltext, updated_state, perturbed_path = full_perturbation_pipeline(network, attractor, node, state)

📌 JUN은(는) 23번째 노드이며, attractor 상태를 1 → 0로 변경했습니다.
📄 Perturbed network saved to: networks/grieco_JUN_OFF.txt

🔧 아래를 BNred.main()에 사용하세요:
BNred.main("networks/grieco_JUN_OFF.txt", "01101010100100001011111011001111100110110110101111110")
❗ 아직 결과 파일이 없습니다: results/grieco_JUN_OFF_01101010100100001011111011001111100110110110101111110/OnePlusProductNet.txt


In [44]:
import os

def prepare_for_bnred(network_path, original_state, mutated_node, mutated_state):
    if mutated_state.upper() == 'ON':
        logic = f"{mutated_node} = {mutated_node} or not {mutated_node}"
        state_bit = '1'
    elif mutated_state.upper() == 'OFF':
        logic = f"{mutated_node} = {mutated_node} and not {mutated_node}"
        state_bit = '0'
    else:
        raise ValueError("mutated_state must be 'ON' or 'OFF'")

    with open(network_path, 'r') as f:
        lines = f.readlines()

    node_order = [line.split('=')[0].strip() for line in lines]
    if mutated_node not in node_order:
        raise ValueError(f"Node '{mutated_node}' not found in network.")
    idx = node_order.index(mutated_node)

    updated_state = list(original_state)
    old_val = updated_state[idx]
    updated_state[idx] = state_bit
    updated_state_str = ''.join(updated_state)
    print(f"📌 {mutated_node}은(는) {idx}번째 노드이며, attractor 상태를 {old_val} → {state_bit}로 변경했습니다.")

    # 저장 경로 설정
    network_dir = os.path.dirname(network_path)
    base_name = os.path.splitext(os.path.basename(network_path))[0]
    perturbed_name = f"{base_name}_{mutated_node}_{mutated_state.upper()}.txt"
    perturbed_path = os.path.join(network_dir, perturbed_name)

    updated_lines = []
    for line in lines:
        if line.strip().startswith(f"{mutated_node} ="):
            updated_lines.append(logic + '\n')
        else:
            updated_lines.append(line)

    with open(perturbed_path, 'w') as f:
        f.writelines(updated_lines)
    print(f"📄 Perturbed network saved to: {perturbed_path}")

    # BNred 실행 문자열 안내
    print(f"\n🔧 아래를 BNred.main()에 사용하세요:")
    print(f"BNred.main(\"{perturbed_path}\", \"{updated_state_str}\")")

    return perturbed_path, updated_state_str


def load_modeltext_after_bnred(perturbed_path, updated_state_str):
    base_name = os.path.splitext(os.path.basename(perturbed_path))[0]
    parent_dir = os.path.dirname(os.path.dirname(perturbed_path))  # 상위 폴더
    result_folder = os.path.join(parent_dir, 'results', f"{base_name}_{updated_state_str}")
    model_path = os.path.join(result_folder, "OnePlusProductNet.txt")

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"❗ BNred 결과 파일이 없습니다: {model_path}")

    with open(model_path, 'r') as f:
        modeltext = f.read()

    print(f"📥 modeltext 로드 완료: {model_path}")
    return modeltext

In [45]:
# Step 1: 네트워크 준비 및 안내 출력
network = "networks/grieco.txt"
attractor = "01101010100100001011111111001111100110110110101111110"
node = "JUN"
state = "OFF"

perturbed_path, updated_state = prepare_for_bnred(network, attractor, node, state)

# 👇 이 사이에 BNred.main(...) 직접 실행할 것!
BNred.main(perturbed_path, updated_state)

# Step 2: BNred 결과 로딩
modeltext = load_modeltext_after_bnred(perturbed_path, updated_state)

📌 JUN은(는) 23번째 노드이며, attractor 상태를 1 → 0로 변경했습니다.
📄 Perturbed network saved to: networks/grieco_JUN_OFF.txt

🔧 아래를 BNred.main()에 사용하세요:
BNred.main("networks/grieco_JUN_OFF.txt", "01101010100100001011111011001111100110110110101111110")


100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 482.39it/s]

AKT = PTEN | ~PDK1
AP1 = (ATF2 & ~JUN) | (~FOS & ~JUN)
ATF2 = JNK | p38
ATM = DNA_damage
Apoptosis = BCL2 & ERK & FOXO3 & p53
BCL2 = AKT | ~CREB
CREB = MSK
DNA_damage = DNA_damage
DUSP1 = CREB
EGFR = GRB2 | ~PKC | (EGFR_stimulus & SPRY)
EGFR_stimulus = EGFR_stimulus
ELK1 = JNK | p38 | ~ERK
ERK = MEK1_2
FGFR3 = FGFR3_stimulus | GRB2 | ~PKC
FGFR3_stimulus = FGFR3_stimulus
FOS = ERK | RSK | (~CREB & ~ELK1)
FOXO3 = AKT & JNK
FRS2 = FGFR3 | GRB2 | ~SPRY
GAB1 = GRB2 | PI3K
GADD45 = SMAD | p53
GRB2 = TGFBR | ~EGFR | ~FRS2
Growth_Arrest = p21
JNK = (MAP3K1_3 & MTK1) | (MAP3K1_3 & TAK1) | (MTK1 & TAK1) | (MAP3K1_3 & ~DUSP1) | (MAP3K1_3 & ~TAOK) | (MTK1 & ~DUSP1) | (MTK1 & ~TAOK) | (TAK1 & ~DUSP1) | (TAK1 & ~TAOK) | (~DUSP1 & ~TAOK)
JUN = True
MAP3K1_3 = RAS
MAX = p38
MDM2 = p14 | (AKT & ~p53)
MEK1_2 = AP1 | PPP2CA | (~MAP3K1_3 & ~RAF)
MSK = p38 | ~ERK
MTK1 = GADD45
MYC = (MAX & MSK) | (MSK & ~AKT)
PDK1 = PI3K
PI3K = GAB1 | (RAS & SOS)
PKC = PLCG
PLCG = EGFR & FGFR3
PPP2CA = p38
PTEN = p53
Proli

In [47]:
modeltext = modeltext.replace("=", "*=").replace(" & ", " and ").replace(" | ", " or ").replace("~ ", "not ")
modeltext = modeltext.strip()

In [48]:
modeltext

'AKT *= PTEN\nAP1 *= \nATF2 *= JNK\nATM *= DNA_damage\nApoptosis *= BCL2 and ERK and FOXO3 and p53\nBCL2 *= AKT\nCREB *= MSK\nDNA_damage *= DNA_damage\nDUSP1 *= CREB\nEGFR *= GRB2\nEGFR_stimulus *= EGFR_stimulus\nELK1 *= JNK\nERK *= MEK1_2\nFGFR3 *= FGFR3_stimulus\nFGFR3_stimulus *= FGFR3_stimulus\nFOS *= ERK\nFOXO3 *= AKT and JNK\nFRS2 *= FGFR3\nGAB1 *= GRB2\nGADD45 *= SMAD\nGRB2 *= TGFBR\nGrowth_Arrest *= p21\nJNK *= (MAP3K1_3 and MTK1)\nJUN *= True\nMAP3K1_3 *= RAS\nMAX *= p38\nMDM2 *= p14\nMEK1_2 *= AP1\nMSK *= p38\nMTK1 *= GADD45\nMYC *= (MAX and MSK)\nPDK1 *= PI3K\nPI3K *= GAB1\nPKC *= PLCG\nPLCG *= EGFR and FGFR3\nPPP2CA *= p38\nPTEN *= p53\nProliferation *= p21\nRAF *= (AKT and ERK and RAS)\nRAS *= SOS\nRSK *= ERK\nSMAD *= TGFBR\nSOS *= GRB2 and RSK\nSPRY *= ERK\nTAK1 *= TGFBR\nTAOK *= ATM\nTGFBR *= TGFBR_stimulus\nTGFBR_stimulus *= TGFBR_stimulus\np14 *= MYC\np21 *= AKT and p53\np38 *= (MAP3K1_3 and MTK1)\np53 *= (MDM2 and p38)\np70 *= ERK'

In [49]:

startTime = time.time()
examNet = cana.boolean_network.BooleanNetwork.from_string_boolean(modeltext)
#print(examNet)

# Mapping nodes
mappindDic = {}
for node in examNet.nodes:
    mappindDic[node.id] = node.name
#print(mappindDic)



# FVSs
FVS_bruteforce = examNet.feedback_vertex_set_driver_nodes(graph='structural', method='bruteforce', max_search=10, keep_self_loops=True) # brutuforce
#print(FVS_bruteforce)
FVS_listList = []
for FVS in FVS_bruteforce:
    FVS_list = []
    for node in FVS:
        FVS_list.append(mappindDic[node])
    FVS_listList.append(FVS_list)
print(FVS_listList)
endTime = time.time() - startTime
#print(endTime)

SyntaxError: unexpected EOF while parsing (<string>, line 1)